# AWS S3 Tutorial with boto3

This notebook demonstrates how to interact with AWS S3 using the boto3 library.

## Prerequisites
1. AWS Account
2. AWS CLI configured with credentials
3. boto3 installed (`pip install boto3`)

In [1]:
import boto3
import json
from datetime import datetime, timedelta

ModuleNotFoundError: No module named 'boto3'

## 1. Connecting to S3

First, we'll create an S3 client using boto3:

In [ ]:
# Create an S3 client
s3 = boto3.client('s3')

# Alternative: using resource instead of client
s3_resource = boto3.resource('s3')

## 2. Basic Operations

### 2.1 Listing Buckets

In [ ]:
response = s3.list_buckets()
for bucket in response['Buckets']:
    print(f'Bucket Name: {bucket["Name"]}')

### 2.2 Creating a Bucket

In [ ]:
def create_bucket(bucket_name, region='us-east-1'):
    try:
        if region == 'us-east-1':
            response = s3.create_bucket(Bucket=bucket_name)
        else:
            response = s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f'Bucket {bucket_name} created successfully')
        return response
    except Exception as e:
        print(f'Error creating bucket: {str(e)}')
        return None

### 2.3 Uploading Files

In [ ]:
def upload_file(file_path, bucket_name, object_name=None):
    if object_name is None:
        object_name = file_path
    
    try:
        response = s3.upload_file(file_path, bucket_name, object_name)
        print(f'File uploaded successfully to {bucket_name}/{object_name}')
        return True
    except Exception as e:
        print(f'Error uploading file: {str(e)}')
        return False

### 2.4 Downloading Files

In [ ]:
def download_file(bucket_name, object_name, file_path):
    try:
        s3.download_file(bucket_name, object_name, file_path)
        print(f'File downloaded successfully to {file_path}')
        return True
    except Exception as e:
        print(f'Error downloading file: {str(e)}')
        return False

## 3. Advanced Operations

### 3.1 Generating Presigned URLs

In [ ]:
def generate_presigned_url(bucket_name, object_name, expiration=3600):
    try:
        url = s3.generate_presigned_url(
            'get_object',
            Params={'Bucket': bucket_name, 'Key': object_name},
            ExpiresIn=expiration
        )
        print(f'Presigned URL: {url}')
        return url
    except Exception as e:
        print(f'Error generating presigned URL: {str(e)}')
        return None

### 3.2 Managing Object Versioning

In [ ]:
def enable_versioning(bucket_name):
    try:
        s3.put_bucket_versioning(
            Bucket=bucket_name,
            VersioningConfiguration={'Status': 'Enabled'}
        )
        print(f'Versioning enabled for bucket {bucket_name}')
        return True
    except Exception as e:
        print(f'Error enabling versioning: {str(e)}')
        return False

## 4. Best Practices and Error Handling

### 4.1 Implementing Retry Logic

In [ ]:
import boto3.s3.transfer as transfer

def upload_with_retry(file_path, bucket_name, object_name=None, max_attempts=3):
    config = transfer.TransferConfig(
        multipart_threshold=1024 * 25,  # 25MB
        max_concurrency=10,
        multipart_chunksize=1024 * 25,
        use_threads=True
    )
    
    for attempt in range(max_attempts):
        try:
            s3.upload_file(
                file_path,
                bucket_name,
                object_name or file_path,
                Config=config
            )
            print(f'File uploaded successfully on attempt {attempt + 1}')
            return True
        except Exception as e:
            if attempt == max_attempts - 1:
                print(f'Final attempt failed: {str(e)}')
                return False
            print(f'Attempt {attempt + 1} failed, retrying...')

## 5. Real-world Example: File Management System

In [ ]:
class S3FileManager:
    def __init__(self, bucket_name, base_path=''):
        self.s3 = boto3.client('s3')
        self.bucket_name = bucket_name
        self.base_path = base_path.strip('/')
    
    def list_files(self, prefix=''):
        full_prefix = f'{self.base_path}/{prefix}'.strip('/')
        response = self.s3.list_objects_v2(
            Bucket=self.bucket_name,
            Prefix=full_prefix
        )
        return [obj['Key'] for obj in response.get('Contents', [])]
    
    def upload_file(self, file_path, object_name=None):
        if not object_name:
            object_name = os.path.basename(file_path)
        full_path = f'{self.base_path}/{object_name}'.strip('/')
        return upload_with_retry(file_path, self.bucket_name, full_path)
    
    def get_presigned_url(self, object_name, expiration=3600):
        full_path = f'{self.base_path}/{object_name}'.strip('/')
        return generate_presigned_url(self.bucket_name, full_path, expiration)

## Usage Example

In [ ]:
# Initialize file manager
file_manager = S3FileManager('your-bucket-name', 'path/to/files')

# List files
files = file_manager.list_files()
print('Files in bucket:', files)

# Upload a file
file_manager.upload_file('local_file.txt', 'remote_file.txt')

# Generate presigned URL
url = file_manager.get_presigned_url('remote_file.txt')
print('Presigned URL:', url)